In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error
import warnings
warnings.filterwarnings('ignore')  # 忽略警告信息

# 设置图片清晰度
plt.rcParams['figure.dpi'] = 300

# 配置中文字体以确保中文正常显示
try:
    plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
except:
    print("中文字体设置失败，将使用默认字体")

# 读取租房和二手房数据
print("=== 数据读取 ===")
df_rent = pd.read_csv('张家口租房合并数据.csv')
df_sale = pd.read_csv('张家口二手房合并数据.csv')

# 输出数据基本形状
print(f"租房数据形状: {df_rent.shape}")
print(f"二手房数据形状: {df_sale.shape}")

# 预览数据结构
print("\n=== 数据预览 ===")
print("租房数据列名:", df_rent.columns.tolist())
print("租房数据前3行:")
print(df_rent.head(3))
print("\n二手房数据列名:", df_sale.columns.tolist())
print("二手房数据前3行:")
print(df_sale.head(3))

# 数据清洗：提取数值特征并计算衍生指标
print("\n=== 数据清洗 ===")

# 处理租房数据：提取面积数值，计算每平方米租金
df_rent['面积_数值'] = df_rent['面积'].str.extract('(\d+\.?\d*)').astype(float)
df_rent['每平方米租金'] = df_rent['租金(元/月)'] / df_rent['面积_数值']

# 处理二手房数据：提取建筑面积、总价数值，计算每平方米房价
df_sale['建筑面积_数值'] = df_sale['建筑面积'].str.extract('(\d+\.?\d*)').astype(float)
df_sale['总价_数值'] = df_sale['总价'].str.extract('(\d+\.?\d*)').astype(float)
df_sale['每平方米房价'] = (df_sale['总价_数值'] * 10000) / df_sale['建筑面积_数值']

# 输出清洗后有效记录数
print("数据清洗完成!")
print(f"租房数据有效记录: {df_rent.dropna(subset=['面积_数值', '每平方米租金']).shape[0]}")
print(f"二手房数据有效记录: {df_sale.dropna(subset=['建筑面积_数值', '每平方米房价']).shape[0]}")

# 定义异常值处理函数（基于四分位距IQR）
def remove_outliers(df, col):
    Q1 = df[col].quantile(0.25)  # 下四分位
    Q3 = df[col].quantile(0.75)  # 上四分位
    IQR = Q3 - Q1  # 四分位距
    lower_bound = Q1 - 1.5 * IQR  # 下界
    upper_bound = Q3 + 1.5 * IQR  # 上界
    return df[(df[col] >= lower_bound) & (df[col] <= upper_bound)].copy()

# 对房价和租金进行异常值处理
df_sale_clean = remove_outliers(df_sale, '每平方米房价')
df_rent_clean = remove_outliers(df_rent, '每平方米租金')

# 输出异常值处理后的记录数
print(f"\n异常值处理完成!")
print(f"清洗后租房数据: {df_rent_clean.shape[0]} 条记录")
print(f"清洗后二手房数据: {df_sale_clean.shape[0]} 条记录")

# 定义建模使用的特征
categorical_feat = ['区域']  # 分类特征：区域
numeric_feat_sale = ['建筑面积_数值']  # 二手房数值特征：建筑面积
numeric_feat_rent = ['面积_数值']  # 租房数值特征：面积

# 输出使用的特征信息
print(f"\n使用的特征:")
print(f"分类特征: {categorical_feat}")
print(f"二手房数值特征: {numeric_feat_sale}")
print(f"租房数值特征: {numeric_feat_rent}")

# 定义特征预处理流程
preprocessor_sale = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_feat),
        ('num', 'passthrough', numeric_feat_sale)
    ])

preprocessor_rent = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_feat),
        ('num', 'passthrough', numeric_feat_rent)
    ])

# 训练房价和租金预测模型
print("\n=== 开始训练模型 ===")

# 房价预测模型（Model 1）
X_sale = df_sale_clean[categorical_feat + numeric_feat_sale]  # 特征数据
y_sale = df_sale_clean['每平方米房价']  # 目标变量

# 构建模型管道（预处理+线性回归）
model_price = Pipeline(steps=[
    ('preprocessor', preprocessor_sale),
    ('regressor', LinearRegression())
])

# 训练模型并生成预测值
model_price.fit(X_sale, y_sale)
df_sale_clean.loc[:, '预测每平方米房价'] = model_price.predict(X_sale)

# 评估模型性能（R²和RMSE）
r2_price = r2_score(y_sale, df_sale_clean['预测每平方米房价'])
rmse_price = np.sqrt(mean_squared_error(y_sale, df_sale_clean['预测每平方米房价']))
print(f"【Model 1：房价预测】")
print(f"R²拟合优度：{r2_price:.4f} | RMSE误差：{rmse_price:.2f}元/m²")

# 租金预测模型（Model 2）
X_rent = df_rent_clean[categorical_feat + numeric_feat_rent]  # 特征数据
y_rent = df_rent_clean['每平方米租金']  # 目标变量

# 构建模型管道
model_rent = Pipeline(steps=[
    ('preprocessor', preprocessor_rent),
    ('regressor', LinearRegression())
])

# 训练模型并生成预测值
model_rent.fit(X_rent, y_rent)
df_rent_clean.loc[:, '预测每平方米租金'] = model_rent.predict(X_rent)

# 评估模型性能
r2_rent = r2_score(y_rent, df_rent_clean['预测每平方米租金'])
rmse_rent = np.sqrt(mean_squared_error(y_rent, df_rent_clean['预测每平方米租金']))
print(f"【Model 2：租金预测】")
print(f"R²拟合优度：{r2_rent:.4f} | RMSE误差：{rmse_rent:.2f}元/m²/月")

# 对所有数据应用模型进行预测
print("\n=== 对所有数据进行预测 ===")

all_X_sale = df_sale[categorical_feat + numeric_feat_sale]
df_sale['预测每平方米房价'] = model_price.predict(all_X_sale)

all_X_rent = df_rent[categorical_feat + numeric_feat_rent]
df_rent['预测每平方米租金'] = model_rent.predict(all_X_rent)

print("预测完成!")

# 计算各区域房价租金比
print("\n=== 计算房价租金比 ===")

# 按区域计算平均租金
rent_by_area = df_rent.groupby('区域')['预测每平方米租金'].mean()

# 计算每个区域的房价租金比（房价/年租金）
area_ratio_data = []
for area in df_sale['区域'].unique():
    if area in rent_by_area.index:
        avg_rent = rent_by_area[area]  # 区域平均月租金
        median_price = df_sale[df_sale['区域'] == area]['预测每平方米房价'].median()  # 区域房价中位数
        price_rent_ratio = median_price / (avg_rent * 12)  # 转换为年租金计算比值
        area_ratio_data.append({'区域': area, '房价租金比': price_rent_ratio})

area_ratio = pd.DataFrame(area_ratio_data)

# 输出各区域房价租金比结果
print("【各区域房价租金比中位数（预测值）】")
print(area_ratio.round(2))

# 可视化各区域房价租金比
print("\n=== 生成可视化图表 ===")

# 创建图形
fig, ax = plt.subplots(figsize=(10, 6))

# 按房价租金比排序
area_ratio_sorted = area_ratio.sort_values('房价租金比', ascending=False)

# 绘制柱状图
bars = ax.bar(area_ratio_sorted['区域'], area_ratio_sorted['房价租金比'], 
              color=['#4e79a7', '#f28e2b', '#e15759', '#76b7b2'], alpha=0.7)

# 添加全球公允价值参考线（≈200）
ax.axhline(y=200, color='red', linestyle='--', linewidth=2, label='全球公允价值（≈200）')

# 为柱状图添加数值标签
for bar in bars:
    height = bar.get_height()
    ax.annotate(f'{height:.1f}',
                xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3),
                textcoords='offset points',
                ha='center', va='bottom',
                fontsize=10, fontweight='bold')

# 设置图表标签和标题
ax.set_xlabel('区域', fontsize=12, fontweight='bold')
ax.set_ylabel('房价租金比', fontsize=12, fontweight='bold')
ax.set_title('张家口各区域房价租金比中位数（线性回归预测）', fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', fontsize=10)
ax.grid(axis='y', alpha=0.3, linestyle='-')

# 调整y轴范围
max_ratio = area_ratio_sorted['房价租金比'].max()
ax.set_ylim(0, max_ratio * 1.2)

# 保存并显示图表
plt.tight_layout()
plt.savefig('zhangjiakou_price_rent_ratio.png', dpi=300, bbox_inches='tight')
plt.show()

# 分析完成提示
print("\n=== 分析完成 ===")
print(f"图表已保存为 'zhangjiakou_price_rent_ratio.png'")

# 模型性能总结表格
results_summary = pd.DataFrame({
    '模型': ['房价预测模型', '租金预测模型'],
    'R²得分': [r2_price, r2_rent],
    'RMSE': [rmse_price, rmse_rent]
})

print("\n模型性能总结:")
print(results_summary.round(4))
    